# EEG DataLoader

The dimensions of the training set are as follows: 4,500 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_train.

The dimensions of the testing set are as follows: 750 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_test.

You can download it from this Google Drive link: [https://drive.google.com/drive/folders/1ykR-mn4d4KfFeeNrfR6UdtebsNRY8PU2?usp=sharing]. 
Please download the data and place it in your data_path at "./data."

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
data_path = './data/'

In [3]:
train_data = np.load(data_path + 'train_data.npy')
test_data = np.load(data_path + 'test_data.npy')
train_label = np.load(data_path + 'train_label.npy')
test_label = np.load(data_path + 'test_label.npy')

train_data_mean = np.mean(train_data)
train_data_std = np.std(train_data)
train_data_normalized = (train_data - train_data_mean) / train_data_std
test_data_normalized = (test_data - train_data_mean) / train_data_std

#To convert the data into PyTorch tensors
x_train_tensor = torch.Tensor(train_data_normalized)
y_train_tensor = torch.LongTensor(train_label)
x_test_tensor = torch.Tensor(test_data_normalized)
y_test_tensor = torch.LongTensor(test_label)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Setting GPU on your computer

In [5]:
train_dataset = TensorDataset(x_train_tensor.to(device), y_train_tensor.to(device)) # input data to Tensor dataloader
train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True, shuffle=True) #  Batch size refers to the number of data sample
test_dataset = TensorDataset(x_test_tensor.to(device), y_test_tensor.to(device))
test_loader = DataLoader(test_dataset, batch_size=64,  drop_last=True,shuffle=False)

# Build simple Deep learning model

In [6]:
class EEGAutoencoderClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EEGAutoencoderClassifier, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(64 * 795, 384), # Input dimention is 64 channel * 795 time point, and use 256 units for first NN layer
            nn.BatchNorm1d(384),
            nn.LeakyReLU(negative_slope=0.05), # Use LeakyReLu function for NN training 
            nn.Dropout(0.05),
            nn.Linear(384, 256), # 256 NN units to 192 units
            nn.BatchNorm1d(256),
            nn.LeakyReLU(negative_slope=0.05),
            nn.Dropout(0.05),
            nn.Linear(256, 192),#  192 NN units to 128 units
            nn.LeakyReLU(negative_slope=0.05)
        )
        self.classifier = nn.Sequential(
            nn.Linear(192, num_classes), # num_classes is 5 (hello,” “help me,” “stop,” “thank you,” and “yes”)
            nn.LogSoftmax(dim=1)  # Use LogSoftmax for multi-class classification
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.encoder(x)
        
        # import pdb;pdb.set_trace()
        x = self.classifier(x)
        return x

In [7]:
num_classes = 5 # setting final output class
model = EEGAutoencoderClassifier(num_classes).to(device) 
criterion = nn.NLLLoss() # Use NLLLoss function to optimize
optimizer = optim.Adam(model.parameters(), lr=0.001) # Setting parameters learning rate = 0.001

In [8]:
num_epochs = 30 # setting training epochs (Number of training iterations)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader: 
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/30, Loss: 1.5980241298675537
Epoch 2/30, Loss: 1.5214102268218994
Epoch 3/30, Loss: 1.495638132095337
Epoch 4/30, Loss: 1.5013048648834229
Epoch 5/30, Loss: 1.406360149383545
Epoch 6/30, Loss: 1.299492359161377
Epoch 7/30, Loss: 1.1739927530288696
Epoch 8/30, Loss: 1.0142018795013428
Epoch 9/30, Loss: 1.047625184059143
Epoch 10/30, Loss: 0.9713567495346069
Epoch 11/30, Loss: 0.6496620774269104
Epoch 12/30, Loss: 0.7342597842216492
Epoch 13/30, Loss: 0.687425434589386
Epoch 14/30, Loss: 0.8571613430976868
Epoch 15/30, Loss: 0.37195613980293274
Epoch 16/30, Loss: 0.5547126531600952
Epoch 17/30, Loss: 0.3435070514678955
Epoch 18/30, Loss: 0.638279914855957
Epoch 19/30, Loss: 0.37743520736694336
Epoch 20/30, Loss: 0.3630385994911194
Epoch 21/30, Loss: 0.2964618504047394
Epoch 22/30, Loss: 0.26617637276649475
Epoch 23/30, Loss: 0.19702193140983582
Epoch 24/30, Loss: 0.2271583527326584
Epoch 25/30, Loss: 0.4271465837955475
Epoch 26/30, Loss: 0.3781319856643677
Epoch 27/30, Loss: 0.18

In [9]:
model.eval() # Evaluate your model
correct = 0
total = 0

with torch.no_grad():
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 49.43%
